*Kind reminder:* Something unclear? Google it!

# Classes

**Bonus topic**: Type vs Class: almost interchangeable, an [article](https://blog.kotlin-academy.com/programmer-dictionary-class-vs-type-vs-object-e6d1f74d1e2e)

##### Classes → @classmethod

[stackoverflow: classmethod vs staticmethod](https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner)

In [ ]:
class A:
    @classmethod
    def foo(cls):
        return cls.__name__

a = A(); b = A()

In [ ]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'foo': <classmethod(<function A.foo at 0x1070ed940>)>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [ ]:
a.__dict__, b.__dict__

({}, {})

In [ ]:
# How to access foo from the class and from the instance?
a.foo(), A.foo()

('A', 'A')

##### Classes → @classmethod → Example

In [ ]:
class A:
    Y = 10

    @classmethod
    def print_y(cls):
        return f"{cls.__name__} has Y={cls.Y}"

class B(A):
    Y = 20
    pass

In [ ]:
b = B()
a = A()
a.print_y(), b.print_y()

('A has Y=10', 'B has Y=20')

In [ ]:
import os
from typing import TypeVar, Type

# Create a generic variable that can be 'Parent', or any subclass.
T = TypeVar('A', bound='Parent')

class A:
    def __init__(self, folder: str, file_name: str):
        self.folder = folder
        self.file_name = file_name
    
    @classmethod
    def from_path(cls: Type[T], path: str) -> T:
        folder, file_name = path.rsplit("/", 1)
        return cls(folder, file_name)

### Dunder methods

#### Classes → Dunder methods → `__getattr__`/`__setattr__`

In [ ]:
import typing as tp
class StorageBox:
    def __init__(self):
        self.test = 1

    def __getattr__(self, x: str) -> tp.Any:
        print(f'Getting the {x} attribute')
        return self.__dict__.get(x, 0)

    def __setattr__(self, x: str, value: tp.Any) -> None:
        print(f'Setting the {x} attribute with value {value}')
        self.__dict__[x] = value

s = StorageBox()
print(s.test)
print("===")
s.test = 2
print("===")
print(s.test)

Setting the test attribute with value 1
1
===
Setting the test attribute with value 2
===
2


In [ ]:
import typing as tp
class StorageBox:

    def __getattr__(self, x: str) -> tp.Any:
        print(f'Getting the {x} attribute')
        return self.__dict__.get(x, 0)
        
    def __getattribute__(self, x: str) -> tp.Any:
        print(f'In getattribute looking for {x} attribute')
        if x != "__dict__" and x.startswith("_"):
            raise ValueError("Access to private attributes is prohibitted.")
        return object.__getattribute__(self, x)

    def __setattr__(self, x: str, value: tp.Any) -> None:
        print(f'Setting the {x} attribute with value {value}')
        self.__dict__[x] = value

    
s = StorageBox()
print(s.test)
print("===")
s.test = 2
print("===")
print(s.test)

In getattribute looking for test attribute
Getting the test attribute
In getattribute looking for __dict__ attribute
0
===
Setting the test attribute with value 2
In getattribute looking for __dict__ attribute
===
In getattribute looking for test attribute
2


In [ ]:
s._x = 123
s.x

Setting the _x attribute with value 123
In getattribute looking for __dict__ attribute
In getattribute looking for x attribute
Getting the x attribute
In getattribute looking for __dict__ attribute


0

##### Classes → Dunder methods → `__hash__` → do not override anything

In [ ]:
class A:
    def __init__(self, x: int):
        self.x = x
        
    def __repr__(self) -> str:
        return f"A({self.x})"

In [ ]:
object.__dict__

mappingproxy({'__new__': <function object.__new__(*args, **kwargs)>,
              '__repr__': <slot wrapper '__repr__' of 'object' objects>,
              '__hash__': <slot wrapper '__hash__' of 'object' objects>,
              '__str__': <slot wrapper '__str__' of 'object' objects>,
              '__getattribute__': <slot wrapper '__getattribute__' of 'object' objects>,
              '__setattr__': <slot wrapper '__setattr__' of 'object' objects>,
              '__delattr__': <slot wrapper '__delattr__' of 'object' objects>,
              '__lt__': <slot wrapper '__lt__' of 'object' objects>,
              '__le__': <slot wrapper '__le__' of 'object' objects>,
              '__eq__': <slot wrapper '__eq__' of 'object' objects>,
              '__ne__': <slot wrapper '__ne__' of 'object' objects>,
              '__gt__': <slot wrapper '__gt__' of 'object' objects>,
              '__ge__': <slot wrapper '__ge__' of 'object' objects>,
              '__init__': <slot wrapper '__init__' of

In [ ]:
s = set()

for i in [1, 1, 2, 2]:
    a = A(i)
    s.add(a)

s

{A(1), A(1), A(2), A(2)}

In [ ]:
a = A(5)
b = A(5)

print(hash(a) == hash(a), a == a)
print(hash(a) == hash(b), a == b)

True True
False False


In [ ]:
A.__hash__

<slot wrapper '__hash__' of 'object' objects>

##### Classes → Dunder methods → `__hash__` → override only `__eq__`

In [1]:
# Override __eq__

class A:
    def __init__(self, x: int):
        self.x = x
        
    def __repr__(self) -> str:
        return f"A({self.x})"
    
    def __eq__(self, other: 'A') -> bool:
        return self.x == other.x

In [2]:
A.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.A.__init__(self, x: int)>,
              '__repr__': <function __main__.A.__repr__(self) -> str>,
              '__eq__': <function __main__.A.__eq__(self, other: 'A') -> bool>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None,
              '__hash__': None})

In [3]:
s = set()

for i in [1, 1, 2, 2]:
    a = A(i)
    s.add(a)

s

TypeError: unhashable type: 'A'

In [ ]:
A.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.A.__init__(self, x: int)>,
              '__repr__': <function __main__.A.__repr__(self) -> str>,
              '__eq__': <function __main__.A.__eq__(self, other: 'A') -> bool>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None,
              '__hash__': None})

##### Classes → Dunder methods → `__hash__` → override only `__hash__`

In [4]:
# Override __hash__

class A:
    def __init__(self, x: int):
        self.x = x
        
    def __repr__(self) -> str:
        return f"A({self.x})"
    
    def __hash__(self) -> int:
        return hash(self.x)

In [5]:
s = set()

for i in [1, 1, 2, 2]:
    a = A(i)
    s.add(a)

s

{A(1), A(1), A(2), A(2)}

##### Classes → Dunder methods → `__hash__` → override `__eq__` и `__hash__`

In [6]:
class A:
    def __init__(self, x: int):
        self.x = x
        
    def __repr__(self) -> str:
        return f"A({self.x})"
    
    def __hash__(self) -> int:
        return hash(self.x)

    def __eq__(self, other: 'A') -> bool:
        return self.x == other.x

In [7]:
s = set()

for i in [1, 1, 2, 2]:
    a = A(i)
    s.add(a)

s

{A(1), A(2)}